In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [9]:
dataset = pd.read_csv('stock_prices.csv')
dataset

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"
...,...,...,...,...,...,...
1253,12/23/2016,790.90,792.74,787.28,789.91,"623,400"
1254,12/27/2016,790.68,797.86,787.66,791.55,"789,100"
1255,12/28/2016,793.70,794.23,783.20,785.05,"1,153,800"
1256,12/29/2016,783.33,785.93,778.92,782.79,"744,300"


In [11]:
training_set = dataset.iloc[:,1:2].values
print(training_set)
print(training_set.shape)

[[325.25]
 [331.27]
 [329.83]
 ...
 [793.7 ]
 [783.33]
 [782.75]]
(1258, 1)


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_training = scaler.fit_transform(training_set)
scaled_training

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [14]:
x_train=[]
y_train=[]
for i in range(60,1258):
    x_train.append(scaled_training[i-60:i,0])
    y_train.append(scaled_training[i,0])

x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

(1198, 60)
(1198,)


In [15]:
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_train.shape

(1198, 60, 1)

In [19]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))


In [20]:
regressor.compile(optimizer='adam',loss='mean_squared_error')
regressor.fit(x_train,y_train,epochs=100, batch_size=30)

Epoch 1/100
40/40 [==============================] - 9s 78ms/step - loss: 0.1254
Epoch 2/100
40/40 [==============================] - 3s 77ms/step - loss: 0.0855
Epoch 3/100
40/40 [==============================] - 3s 77ms/step - loss: 0.0826
Epoch 4/100
40/40 [==============================] - 3s 78ms/step - loss: 0.0807
Epoch 5/100
40/40 [==============================] - 3s 78ms/step - loss: 0.0803
Epoch 6/100
40/40 [==============================] - 4s 88ms/step - loss: 0.0797
Epoch 7/100
40/40 [==============================] - 4s 95ms/step - loss: 0.0788
Epoch 8/100
40/40 [==============================] - 4s 90ms/step - loss: 0.0781
Epoch 9/100
40/40 [==============================] - 23s 592ms/step - loss: 0.0783
Epoch 10/100
40/40 [==============================] - 3s 79ms/step - loss: 0.0790
Epoch 11/100
40/40 [==============================] - 51s 1s/step - loss: 0.0769
Epoch 12/100
40/40 [==============================] - 6s 148ms/step - loss: 0.0786
Epoch 13/100
40/40 [===

In [ ]:
dataset_test = pd.read_csv("stock_prices.csv")
actual_stock_price = dataset_test.iloc[:,1:2].values

In [ ]:
dataset_total = pd.concat((dataset['Open'],dataset_test['Open'],))